In [20]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Input, Dense, Dropout

from sklearn.model_selection import train_test_split

In [21]:
def generate_time_series(sample_size, n_steps):
 freq1, freq2, offsets1, offsets2 = np.random.rand(4, sample_size, 1)
 time = np.linspace(0, 1, n_steps)
 series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10)) #wave1+
 series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) #wave2+
 series += 0.1 * (np.random.rand(sample_size, n_steps) - 0.5) #noise
 return series[..., np.newaxis].astype(np.float32)

In [22]:
dataset = generate_time_series(10000,51)
dataset.shape

(10000, 51, 1)

In [23]:
df = pd.DataFrame(dataset.squeeze())
df.head()

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,-0.127093,-0.223824,-0.198700,-0.201996,-0.250461,-0.232255,-0.389566,-0.459475,-0.559557,-0.659512,...,0.026051,0.225891,0.477400,0.587549,0.712466,0.648162,0.544950,0.321964,0.264029,0.135570
1,-0.345368,-0.268462,-0.296515,-0.356132,-0.260629,-0.177431,0.113936,0.309156,0.610167,0.657162,...,-0.162661,-0.293219,-0.475181,-0.641415,-0.626319,-0.585295,-0.323810,-0.015371,0.199954,0.261965
2,0.706664,0.606987,0.477661,0.284608,0.074641,-0.082206,-0.181075,-0.274538,-0.226139,-0.171299,...,-0.103793,-0.055984,-0.110767,-0.034440,0.037951,0.208680,0.351388,0.511884,0.689543,0.735335
3,0.289033,-0.013773,-0.232592,-0.452892,-0.536335,-0.562836,-0.454764,-0.286151,-0.183272,-0.063258,...,-0.413934,-0.575517,-0.638992,-0.596422,-0.368493,-0.055457,0.241691,0.465710,0.603824,0.648190
4,-0.676177,-0.637123,-0.598719,-0.509309,-0.317312,-0.136338,0.131101,0.309805,0.329757,0.399872,...,-0.414043,-0.330033,-0.245616,-0.166715,-0.094610,-0.059564,-0.037427,0.086207,0.151655,0.270065


In [24]:
x = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [25]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.1,random_state=42)

In [28]:
model1 = Sequential([
    Input(shape=(50,1)),
    Dense(128,activation='relu'),
    Dense(64,activation='relu'),
    Dense(32,activation='relu'),
    Dense(1)
])

In [29]:
model1.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 50, 128)        │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 50, 64)         │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 50, 32)         │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 50, 1)          │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,625 (41.50 KB)

 Trainable params: 10,625 (41.50 KB)

 Non-trainable params: 0 (0.00 B)

In [31]:
model2 = Sequential([
    Input(shape=(50,1)),
    SimpleRNN(units=50,return_sequences=True),
    Dense(1)
])

In [32]:
model2.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 50, 50)         │         2,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 50, 1)          │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,651 (10.36 KB)

 Trainable params: 2,651 (10.36 KB)

 Non-trainable params: 0 (0.00 B)

In [33]:
model3 = Sequential([
    Input(shape=(50,1)),
    SimpleRNN(units=50, return_sequences=True),
    SimpleRNN(units=50, return_sequences=True),
    Dense(1)
])

In [34]:
model3.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn_1 (SimpleRNN)        │ (None, 50, 50)         │         2,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_2 (SimpleRNN)        │ (None, 50, 50)         │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 50, 1)          │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,701 (30.08 KB)

 Trainable params: 7,701 (30.08 KB)

 Non-trainable params: 0 (0.00 B)

In [37]:
model_dict = {"FCNN":model1,"RNN1":model2,"RNN2":model3}
results=[]

for model_name, model in model_dict.items():
    print(f"Training model: {model_name}")
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
    print(model.summary())
    history = model.fit(x_train, y_train, epochs=25, batch_size=32, validation_split=2/9)
    test_loss, test_mse = model.evaluate(x_test, y_test)

    results.append({
            "model_name": model_name,
            "mse": history.history['mean_squared_error'][-1],
            "test_mse": test_mse
        })

    # Convert results to a DataFrame
    results_df = pd.DataFrame(results)    

Training model: FCNN


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 50, 128)        │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 50, 64)         │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 50, 32)         │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 50, 1)          │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,625 (41.50 KB)

 Trainable params: 10,625 (41.50 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn_1 (SimpleRNN)        │ (None, 50, 50)         │         2,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_2 (SimpleRNN)        │ (None, 50, 50)         │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 50, 1)          │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,701 (30.08 KB)

 Trainable params: 7,701 (30.08 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/25
219/219 ━━━━━━━━━━━━━━━━━━━━ 7s 22ms/step - loss: 0.1487 - mean_squared_error: 0.1487 - val_loss: 0.1123 - val_mean_squared_error: 0.1123
Epoch 2/25
219/219 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 0.1057 - mean_squared_error: 0.1057 - val_loss: 0.0944 - val_mean_squared_error: 0.0944
Epoch 3/25
219/219 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - loss: 0.0891 - mean_squared_error: 0.0891 - val_loss: 0.0869 - val_mean_squared_error: 0.0869
Epoch 4/25
219/219 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 0.0841 - mean_squared_error: 0.0841 - val_loss: 0.0806 - val_mean_squared_error: 0.0806
Epoch 5/25
219/219 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 0.0807 - mean_squared_error: 0.0807 - val_loss: 0.0792 - val_mean_squared_error: 0.0792
Epoch 6/25
219/219 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 0.0790 - mean_squared_error: 0.0790 - val_loss: 0.0798 - val_mean_squared_error: 0.0798
Epoch 7/25
219/219 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 0.0764 - mean_squared_error: 0.0764 - va

In [38]:
results_df = pd.DataFrame(results)

In [39]:
results_df

,model_name,mse,test_mse
0,FCNN,0.145344,0.140129
1,RNN1,0.070326,0.068038
2,RNN2,0.066433,0.062618
